In [1]:
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import networkx as nx
from string import punctuation
import nltk
nltk.download('stopwords')
import io

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
def data_summary(data,top_n):
    stop_words = set(stopwords.words("english"))
    summary=[]
    for X in data:

        #----------------------------------------------------
        #reading sentence
        
        article = nltk.tokenize.sent_tokenize(X)
        sentences = []
        
        for sent in article:

            sentences.append(sent.replace("[^a-zA-Z]", " ").split(" "))
        sentences.pop() 
        
        #print(sentences)
        #----------------------------------------------------
        #sentence similarity

        def sentence_similarity(sent1, sent2, stopwords=None):
            if stopwords is None:
                stopwords = []

            sent1 = [w.lower() for w in sent1]
            sent2 = [w.lower() for w in sent2]
            #print(sent1,sent2)
            all_words = list(set(sent1 + sent2))

            vector1 = [0] * len(all_words)
            vector2 = [0] * len(all_words)

            # build the vector for the first sentence
            for w in sent1:
                if w in stopwords:
                    continue
                vector1[all_words.index(w)] += 1

            # build the vector for the second sentence
            for w in sent2:
                if w in stopwords:
                    continue
                vector2[all_words.index(w)] += 1
            #print(all_words)
            #print(len(all_words))
            return 1 - cosine_distance(vector1, vector2)

        #print("sentence_similarity :",sentence_similarity,'\n','-'*50)
            

        #-----------------------------------------------------------------------------
        #similarity matrix

        similarity_matrix = np.zeros((len(sentences), len(sentences)))
        for idx1 in range(len(sentences)):
                for idx2 in range(len(sentences)):
                    if idx1 == idx2: #ignore if both are same sentences
                        continue 
                    similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)


        #print("similarity matrix :\n",similarity_matrix,'\n','-'*50)

        #------------------------------------------------------------------------------
        #ranking sentence 

        sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(sentence_similarity_graph)
        ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True) 
        #print("ranked_sentence :",ranked_sentence,'\n','-'*50)

        #--------------------------------------------------------------------------------
        #summarize text
        summarize_text = []


        #top_n = 15
        for i in range(top_n):
            if X != '-' :
                summarize_text.append(" ".join(ranked_sentence[i][1]))
            else :
                summarize_text.append('-')
        #print("Summarize Text: \n", ". ".join(summarize_text))
        #print('-'*110)

        summary.append(". ".join(summarize_text))
    
    return summary

### ----------------------------------------------------------------------------------------------------------------------------------------------------------------

## Testing with real news data

In [107]:
Article = '''Data science and big data are often viewed in concert, but data science can be used to extract value from data of all sizes, whether structured, unstructured, or semi-structured. Of course, big data is useful to data scientists in many cases, because the more data you have, the more parameters you can include in a given model.

"With big data, you're not necessarily bound to the dimensionality constraints of small data," Hunt says. "Big data does help in certain aspects, but more isn't always better. If you take the stock market and try to fit it to a line, it's not going to work. But maybe, if you only look at it for a day or two, you can set it to a line."'''

In [108]:
news1_summary = data_summary(pd.Series([Article]),3)
news1_summary

['Of course, big data is useful to data scientists in many cases, because the more data you have, the more parameters you can include in a given model.. Data science and big data are often viewed in concert, but data science can be used to extract value from data of all sizes, whether structured, unstructured, or semi-structured.. "Big data does help in certain aspects, but more isn\'t always better.']